#Matrix Vector Multiplication

First we will look into the serial version of code that does matrix vector multiplication.

In [1]:
%%writefile matvecSerial.c
#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include<time.h>
int n=30000;
void dotp (int *d, int r[n],int k[n], int n)
{
  int i,j;
  for(i=0;i<n;i++)
  for(j=0;j<n;j++)
  k[i]=k[i]+(d[n*i+j]*r[j]);
//printf("this is k in func %lf\n",k);


}

void main()
{
  int n=30000;int i,j;
  int *a; int b[n];int k[n];int c;
  a=(int*)malloc(n*n*sizeof(int));

  for( i=0;i<n;i++)
  {
  b[i]=abs(rand()%7+3);
  k[i]=0;
  for(j=0;j<n;j++)
  {a[n*i+j]=abs(rand()%4+1);}

//  printf("%d  %d\n",a[i],b[i]);
  }
 double total_time=0.000;
  	clock_t begin=clock();
dotp(a,b,k,n);
//printf("%d\n",c );
clock_t end =clock();
total_time += ((double) (end - begin)) / CLOCKS_PER_SEC;
printf(" time %lf\n",total_time );
}


Writing matvecSerial.c


In [ ]:
!ls

matvecSerial.c	sample_data


In [ ]:
!gcc --version
!gcc matvecSerial.c -o matvecSerial

gcc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!ls
!./matvecSerial

matvecSerial  matvecSerial.c  sample_data
 time 4.194886


#Matrix Vector multiplication on GPUs

Now we will see how we can port this code to do matrix vector multiplication on GPUs using CUDA.

Firstly, we need to change the runtime to connect with GPUs.
Go to Runtime>>Change runtime type
and select the T4 GPU.

Next, we will install and load nvhpc for cuda compilers.

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter.plugin


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ukspnu3v
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ukspnu3v
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-hm3a0cqb/wheels/7d/b9/66/459b9938664e6a93d1a85323ec52f7e51cd7265d253410a7d8
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp2np_tk70".


In [3]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Tue Nov 11 06:56:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8       

In [2]:
%%writefile matvecparallel.cu

#include <stdio.h>
#include <cuda.h>
#include <stdlib.h>
#include <time.h>
//int n=2560; int u=16;
__global__ void mvm (int *a, int *x, int *b, int n,int u)
{
  int j=blockIdx.y * blockDim.y + threadIdx.y;
  int i=blockIdx.x * blockDim.x + threadIdx.x;
  int ind= i+gridDim.x*u*j;
  if(ind<n)
  {
    int l;int m=(ind*n);
    *(b+ind)=0;
    for(l=0;l<n;l++)
    *(b+ind)=(*(b+ind))+(*(a+m+l))*(*(x+l));
  }
__syncthreads();
}

void launchmvm(int *ah, int*xh, int *ch, int n, int u)
{
  int*ad;int*xd;int*cd;
  int size=n*sizeof(int);
  int size2=n*n*sizeof(int);
  cudaMalloc((void**)&ad,size2);
  cudaMalloc((void**)&xd,size);
  cudaMalloc((void**)&cd,size);
  cudaMemcpy(ad,ah,size2,cudaMemcpyHostToDevice);
  cudaMemcpy(xd,xh,size,cudaMemcpyHostToDevice);

  int d=ceil(n/256);
  dim3 dimGrid(1,d);
  dim3 dimBlock(16, 16);

  mvm<<<dimGrid,dimBlock>>>(ad,xd,cd,n,u);


  cudaMemcpy(ch,cd,size,cudaMemcpyDeviceToHost);
  cudaFree(ad);
  cudaFree(xd);
  cudaFree(cd);
}
int main ()
{

  int n=30000; int u=16;
  int *ah;
  ah=(int*)malloc(n*n*sizeof(int));
   int xh[n]; int ch[n];
  for(int i=0;i<n;i++)
  {
    xh[i]=rand()%20+1;
    ch[i]=0;
    for(int j=0;j<n;j++)
    {
    ah[n*i+j]=rand()%9+1;
    }
  }
  float time;
  double total_time=0.000;
  clock_t begin=clock();
launchmvm(ah,xh,ch,n,u);
clock_t end =clock();
total_time += ((double) (end - begin)) / CLOCKS_PER_SEC;
printf("\nTime taken is %f \n",total_time);


}


Writing matvecparallel.cu


In [5]:
!ls

matvecparallel.cu  matvecparallelShared.cu  sample_data


Compiling with nvcc for T4 compute capability of 7.5 hence 75 appears in the architecture related flags.

In [ ]:
!nvcc -arch=sm_86 -gencode=arch=compute_86,code=sm_86 matvecparallel.cu -o matvecparallel


matvecparallel.cu(61): warning #177-D: variable "time" was declared but never referenced
    float time;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



Running the code on GPU!!!

In [7]:
!./matvecparallel


Time taken is 0.234689 
